# 02. Deep Neural Network Baseline Model

**Prediksi Risiko Konsumsi Narkoba Menggunakan Deep Neural Network**

---

## 📋 Deskripsi

Notebook ini membangun **baseline Deep Neural Network (DNN)** untuk prediksi risiko konsumsi narkoba. Model ini akan menjadi baseline untuk dibandingkan dengan Random Forest (UTS) dan model DNN teroptimasi.

**Tujuan:**
1. Build baseline DNN architecture
2. Train model dengan early stopping
3. Evaluate performance
4. Compare dengan Random Forest baseline

**Author:** Muhammad Ichsan Junaedi & Amanda Wijayanti  
**Date:** December 2024  
**Institution:** STMIK TAZKIA

## 1. Setup & Import Libraries

In [4]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pickle
from datetime import datetime

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2

# Sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    roc_curve, precision_recall_curve, auc,
    cohen_kappa_score, matthews_corrcoef, balanced_accuracy_score
)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

TensorFlow Version: 2.20.0
Keras Version: 3.12.0
GPU Available: []


In [5]:
# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Increase plot quality
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

---
## 2. Load Preprocessed Data


In [7]:
# Load preprocessed data from 01_Data_Preparation.ipynb
print("Loading preprocessed data...")

# Load training data
X_train = pd.read_csv('../data/processed/X_train.csv')
y_train = pd.read_csv('../data/processed/y_train.csv').values.ravel()

# Load test data
X_test = pd.read_csv('../data/processed/X_test.csv')
y_test = pd.read_csv('../data/processed/y_test.csv').values.ravel()

print(f"✅ Data loaded successfully!")
print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nFeatures: {X_train.shape[1]}")
print(f"Training samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

Loading preprocessed data...
✅ Data loaded successfully!

Training set: (1508, 24)
Test set: (377, 24)

Features: 24
Training samples: 1508
Test samples: 377


In [8]:
# Check class distribution
print("\n" + "="*60)
print("CLASS DISTRIBUTION")
print("="*60)

train_class_dist = pd.Series(y_train).value_counts()
test_class_dist = pd.Series(y_test).value_counts()

print("\nTraining Set:")
print(f"  Non-User (0): {train_class_dist[0]} ({train_class_dist[0]/len(y_train)*100:.2f}%)")
print(f"  User (1): {train_class_dist[1]} ({train_class_dist[1]/len(y_train)*100:.2f}%)")

print("\nTest Set:")
print(f"  Non-User (0): {test_class_dist[0]} ({test_class_dist[0]/len(y_test)*100:.2f}%)")
print(f"  User (1): {test_class_dist[1]} ({test_class_dist[1]/len(y_test)*100:.2f}%)")


CLASS DISTRIBUTION

Training Set:
  Non-User (0): 568 (37.67%)
  User (1): 940 (62.33%)

Test Set:
  Non-User (0): 142 (37.67%)
  User (1): 235 (62.33%)


---
## 3. Feature Scaling for DNN

**CRITICAL:** Deep Neural Networks sangat sensitif terhadap scale input. perlu standardize semua features.

In [9]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit on training data and transform both train and test
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for consistency
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("✅ Feature scaling completed!")
print(f"\nScaled Training Set Shape: {X_train_scaled.shape}")
print(f"Scaled Test Set Shape: {X_test_scaled.shape}")

# Verify scaling
print("\n" + "="*60)
print("SCALING VERIFICATION")
print("="*60)
print(f"\nTraining Set:")
print(f"  Mean: {X_train_scaled.mean().mean():.6f} (should be ≈ 0)")
print(f"  Std: {X_train_scaled.std().mean():.6f} (should be ≈ 1)")
print(f"\nTest Set:")
print(f"  Mean: {X_test_scaled.mean().mean():.6f}")
print(f"  Std: {X_test_scaled.std().mean():.6f}")

✅ Feature scaling completed!

Scaled Training Set Shape: (1508, 24)
Scaled Test Set Shape: (377, 24)

SCALING VERIFICATION

Training Set:
  Mean: 0.000000 (should be ≈ 0)
  Std: 1.000332 (should be ≈ 1)

Test Set:
  Mean: -0.002271
  Std: 0.945033


In [10]:
# Save scaler for future use
with open('../models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("✅ Scaler saved to: ../models/scaler.pkl")

✅ Scaler saved to: ../models/scaler.pkl


---
## 4. Build Baseline DNN Architecture

**Architecture Philosophy:**
- **Moderate depth** (4 hidden layers) - balance complexity vs overfitting
- **Pyramidal structure** - neurons decreasing: 128 → 64 → 32 → 16
- **Regularization** - Dropout for generalization
- **Activation** - ReLU for hidden layers, Sigmoid for output
- **Output** - Single neuron with sigmoid for binary classification

In [11]:
# Get input dimension
input_dim = X_train_scaled.shape[1]
print(f"Input Dimension: {input_dim} features")

Input Dimension: 24 features


In [12]:
def build_baseline_dnn(input_dim):
    """
    Build baseline Deep Neural Network architecture
    
    Architecture:
    - Input Layer: input_dim neurons
    - Hidden Layer 1: 128 neurons, ReLU, Dropout(0.3)
    - Hidden Layer 2: 64 neurons, ReLU, Dropout(0.3)
    - Hidden Layer 3: 32 neurons, ReLU, Dropout(0.2)
    - Hidden Layer 4: 16 neurons, ReLU, Dropout(0.2)
    - Output Layer: 1 neuron, Sigmoid
    
    Args:
        input_dim (int): Number of input features
    
    Returns:
        keras.Model: Compiled DNN model
    """
    model = Sequential([
        # Input Layer
        Dense(128, activation='relu', input_dim=input_dim, name='input_layer'),
        Dropout(0.3, name='dropout_1'),
        
        # Hidden Layer 2
        Dense(64, activation='relu', name='hidden_layer_2'),
        Dropout(0.3, name='dropout_2'),
        
        # Hidden Layer 3
        Dense(32, activation='relu', name='hidden_layer_3'),
        Dropout(0.2, name='dropout_3'),
        
        # Hidden Layer 4
        Dense(16, activation='relu', name='hidden_layer_4'),
        Dropout(0.2, name='dropout_4'),
        
        # Output Layer
        Dense(1, activation='sigmoid', name='output_layer')
    ])
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.AUC(name='auc'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )
    
    return model

In [14]:
# Build model
baseline_model = build_baseline_dnn(input_dim)

# Display model architecture
print("\n" + "="*60)
print("BASELINE DNN ARCHITECTURE")
print("="*60)
baseline_model.summary()


BASELINE DNN ARCHITECTURE


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (Dense)             │ (None, 128)            │         3,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_2 (Dense)          │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_3 (Dense)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_4 (Dense)          │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,081 (55.00 KB)

 Trainable params: 14,081 (55.00 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# Count total parameters
trainable_params = np.sum([np.prod(v.get_shape()) for v in baseline_model.trainable_weights])
print(f"\n📊 Total Trainable Parameters: {trainable_params:,}")

AttributeError: 'Variable' object has no attribute 'get_shape'